* python -m venv python
* python\Scripts\activate
* python -m pip install --upgrade pip setuptools wheel
* python -m pip install jupyterlab
* python -m pip install avro-python3
* python -m pip install snappy
* python -m pip install python-snappy
* python -m pip install pandas
* python -m pip install pandavro

In [1]:
import os
print(os.getpid())

104912


In [2]:
import datetime
import dateutil
import time
import uuid

import json
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [3]:
schema = {
  "type": "record",
  "name": "phone_book",
  "fields": [
    {
      "name": "identify",
      "type": "long"
    },
    {
      "name": "name",
      "type": "string"
    },
    {
      "name": "created_at",
      "type": {
        "type": "long",
        "logicalType": "local-timestamp-micros"
      }
    },
    {
      "name": "cell_phone",
      "type": [
        "null",
        {
          "type": "array",
          "items": {
            "name": "record.cell_phone",
            "type": "record",
            "fields": [
              {
                "name": "ddd",
                "type": "int"
              },
              {
                "name": "number",
                "type": "long"
              }
            ]
          }
        }
      ]
    },
    {
      "name": "code",
      "type": {
        "type": "string",
        "logicalType": "uuid"
      }
    }
  ]
}
schema_parsed = avro.schema.Parse(json.dumps(schema))

In [4]:
df = open("example.avro", "wb")
writer = DataFileWriter(df, DatumWriter(), schema_parsed, codec="snappy")

In [5]:
for i in range(1000000):
    writer.append(
        {
            "identify": 1,
            "name": "Danilo",
            "created_at": int((datetime.datetime.today() - datetime.datetime(1970,1,1)).total_seconds() * 1000000),
            "cell_phone": [{"ddd": 11, "number": 24752569},{"ddd": 11, "number": 247525698}],
            "code": str(uuid.uuid4())
        }
    )
    writer.append(
        {
            "identify": 1,
            "name": "Danilo",
            "created_at": int((datetime.datetime.today() - datetime.datetime(1970,1,1)).total_seconds() * 1000000),
            "cell_phone": [],
            "code": str(uuid.uuid4())
        }
    )

In [6]:
writer.close()

In [ ]:
df = open('example.avro', 'rb')
reader = DataFileReader(df, DatumReader())
for user in reader:
    print(user)
    time.sleep(2)
df.close()

In [ ]:
import pandavro
df = pandavro.read_avro('example.avro')
df.infor()